In [1]:
print('hello world')

hello world


In [2]:
class Car:
    def __init__(self, name, speed, distance, time):
        self.name = name
        self.speed = speed
        self.distance = distance
        self.time = time
        self.rank=-10
    
    def move(self):
        print('The car is moving at', self.speed, 'km/h')
    
    def stop(self):
        print('The car has stopped')
    
    def detail(self):
        print('The car is a', self.name, 'and is moving at', self.speed, 'km/h')

In [3]:
car1=Car('BMW', 10, 100, 10)
car2=Car('Audi', 10, 100, 10)
car3=Car('Benz', 10, 100, 10)

cars=[car1, car2, car3]

timeout=2
speed_rate=0.1

for i in range(len(cars)-1):
    car_A=cars[i]
    car_B=cars[i+1]
    car_B.speed=car_A.speed
    while(car_B.time-car_A.time<=timeout):
        car_B.speed=car_B.speed-speed_rate
        car_B.time=car_B.distance/car_B.speed

print(car1.detail(), car2.detail(), car3.detail())

The car is a BMW and is moving at 10 km/h
The car is a Audi and is moving at 8.300000000000006 km/h
The car is a Benz and is moving at 7.10000000000001 km/h
None None None


In [9]:
import numpy as np

class Car:
    def __init__(self, name, speed, distance, time):
        self.name = name
        self.speed = speed
        self.distance = distance
        self.time = time
        self.rank = -10
    
    def move(self):
        print('The car', self.name, 'is moving at', self.speed, 'km/h')
    
    def stop(self):
        print('The car', self.name, 'has stopped')
    
    def detail(self):
        print('The car', self.name, 'is moving at', self.speed, 'km/h')


class QLearningAgent:
    def __init__(self, num_actions, learning_rate=0.1, discount_factor=0.9, exploration_rate=0.1):
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.num_actions = num_actions
        self.q_values = {}

    def get_q_value(self, state, action):
        if (state, action) not in self.q_values:
            self.q_values[(state, action)] = 0
        return self.q_values[(state, action)]

    def choose_action(self, state):
        if np.random.uniform(0, 1) < self.exploration_rate:
            return np.random.choice(self.num_actions)
        else:
            q_values = [self.get_q_value(state, action) for action in range(self.num_actions)]
            return np.argmax(q_values)

    def update_q_value(self, state, action, reward, next_state):
        best_next_action = max(range(self.num_actions), key=lambda next_action: self.get_q_value(next_state, next_action))
        td_target = reward + self.discount_factor * self.get_q_value(next_state, best_next_action)
        td_error = td_target - self.get_q_value(state, action)
        self.q_values[(state, action)] += self.learning_rate * td_error


def simulate_environment(cars, agent, timeout, speed_rate):
    for i in range(len(cars) - 1):
        car_A = cars[i]
        car_B = cars[i + 1]
        car_B.speed = car_A.speed
        while (car_B.time - car_A.time <= timeout):
            state = (car_B.distance, car_B.speed, car_B.time)
            action = agent.choose_action(state)
            if action == 0:  # Decelerate
                car_B.speed -= speed_rate
            car_B.time = car_B.distance / car_B.speed
            next_state = (car_B.distance, car_B.speed, car_B.time)
            reward = -1 if action == 0 else 0  # Penalize deceleration
            agent.update_q_value(state, action, reward, next_state)


car1 = Car('BMW', 10, 100, 10)
car2 = Car('Audi', 10, 100, 10)
car3 = Car('Benz', 10, 100, 10)
car4 = Car('Benz', 10, 100, 10)

cars = [car1, car2, car3, car4]

timeout = 2
speed_rate = 0.1

agent = QLearningAgent(num_actions=2)  # 2 actions: 0 for decelerate, 1 for maintain speed

simulate_environment(cars, agent, timeout, speed_rate)

print("Final speeds after collision avoidance:")
for car in cars:
    car.detail()


Final speeds after collision avoidance:
The car BMW is moving at 10 km/h
The car Audi is moving at 8.300000000000006 km/h
The car Benz is moving at 7.10000000000001 km/h
The car Benz is moving at 6.2000000000000135 km/h
